## 取得年報

### 1️⃣  匯入套件

In [ ]:
import requests
from bs4 import BeautifulSoup

### 2️⃣ 建立函式-取得年報

In [ ]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  # 建立 POST 請求的表單
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data)
    # 取得回應後擷取檔案名稱
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")

  # 建立第二個 POST 請求的表單
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1 # 檔案名稱
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    # 取得 PDF 連結
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")

  # 發送 GET 請求
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)

    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")


### 3️⃣ 呼叫函式

In [ ]:
annual_report('2330','112')

2022_2330_20230606F04.pdf
/pdf/2022_2330_20230606F04_20240421_234318.pdf
OK


## 年報問答

###4️⃣ 安裝及匯入相關套件

In [ ]:
!pip install langchain openai tiktoken pdfplumber==0.10.2 faiss-cpu
import os
import getpass

# LangChain 相關套件
from langchain.document_loaders import PDFPlumberLoader             # 讀取 PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 字串切割
from langchain.embeddings import OpenAIEmbeddings                   # 嵌入方法
from langchain.vectorstores import FAISS                            # 向量資料庫
from langchain.chat_models import ChatOpenAI                        # GPT 模型

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 5️⃣ 建立 LangChain 的 OpenAI 模型

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass("請輸入金鑰：")
llm_16k = ChatOpenAI(model="gpt-3.5-turbo-1106")

請輸入金鑰：··········


### 6️⃣ 建立向量資料庫

In [ ]:
def pdf_loader(file,size,overlap):
  loader = PDFPlumberLoader(file)
  doc = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(
                          chunk_size=size,
                          chunk_overlap=overlap)
  new_doc = text_splitter.split_documents(doc)
  db = FAISS.from_documents(new_doc, OpenAIEmbeddings())
  return db

db = pdf_loader('/content/112_2330.pdf',500, 50)

### 7️⃣ 查詢相關資料

In [ ]:
query = "公司是否有明確的成長或創新策略?"
docs = db.similarity_search(query,k=3)
for i in docs:
    print(i.page_content)
    print('_________')

5.4.1 客戶
提升同仁問題解決與創新的能力，以維持台積公司競爭
優勢並達到客戶滿意的雙贏目標。除了公司內部跨組織 台積公司的客戶遍布全球，產品種類眾多，在半導 最近二年度佔全年度合併營業收入淨額10%以上之客戶資料
的學習交流外，台積公司亦透過「台灣持續改善活動競 體產業的各個領域中表現傑出。客戶包括有無晶
單位：新台幣仟元
賽」，跨產業分享改善手法，期望能以台積公司的經驗分 圓廠設計公司、系統公司和整合元件製造商，例 民國111年 民國110年
名稱
享，促進其他在地產業的發展與進步，並藉由觀摩其他 如：Advanced Micro Devices, Inc.、Amazon Web 佔全年度營業收入 佔全年度營業收入
金額 與發行人之關係 金額 與發行人之關係
淨額比率（%） 淨額比率（%）
產業的改善方法，提升同仁問題解決與創新能力。民國 Services, Inc.、Broadcom Inc.、Intel Corporation、聯
甲客戶 529,649,200 23% 無 405,402,955 26% 無
_________
能對台積公司的營運與財務表現產生負面影響。 台積公司的營運成長及產能的持續擴充計劃，仰賴來自 的營運成本，如果這些專利主張實體成功阻攔本公司所
若台積公司無法克服上述挑戰，則可能對公司業務、財 市場中少數的供應商以得到充分的設備及相關服務，因 提供的產品及服務之交易，亦可能會嚴重干擾本公司的
務狀況及經營結果造成負面影響。 採購集中之風險及因應措施 此可能面臨供應商供給有限且交貨期長的情況，為了優 營運。再者，隨著台積公司的製造營運擴大至部分國外區
●原物料 化供應鏈管理，台積公司針對交貨前置期進行評估及預 域，本公司就智慧財產權被侵害之風險管理亦面臨更高
銷售集中之風險及因應措施 台積公司需及時並以合理的市場價格獲得生產營運所需 測，以期能夠盡量降低供應鏈風險對營運成本的影響， 的挑戰。縱使本公司努力採取有力的措施以減輕在該等
近年來，台積公司的客戶群與客戶的業務性質出現顯著 之足夠原物料，例如矽晶圓、製程用氣體、化學原料、光 台積公司也與各供應商持續擬定多項商務合作模式與 新區域內智慧財產權被侵害的風險，仍無法保證本公司
_________
持其未來發展優勢。
我們將持續致力於優化製造營運（包括「數位化」我們的晶圓廠）來提

### 8️⃣  匯入問答相關套件

In [ ]:
from langchain.prompts import ChatPromptTemplate # 提示模板
from langchain.chains import RetrievalQA         # 問答模組

### 9️⃣  建立函式-問答程式

In [ ]:
# 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,\
      如果有明確數據或技術(產品)名稱可以用數據或名稱回答,\
      回答以繁體中文和台灣用語為主。\
      {context}"),
    ("human", "{question}")])

# 建立問答函式
def question_and_answer(question):
    qa = RetrievalQA.from_llm(llm=llm_16k,
                              prompt=prompt,
                              return_source_documents=True,
                              retriever=db.as_retriever(
                                  search_kwargs={'k':10}))
    result = qa(question)
    return result

### 🔟 建立迴圈進行問答

輸入範例
```
近一年的營收狀況?
目前的產品有哪些？
目前正在開發的項目是?
未來一年的計畫？
```

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result=question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

輸入問題:近一年的營收狀況?
根據提供的資料，台積公司的營收狀況有所增長。營業收入淨額在最近一年增加了43%，這主要是因為平均銷售單價提高、晶圓出貨量增加和受匯率變動增加所致。同時，營業毛利也增加了65%，顯示公司的營業效益有所提升。
_________
輸入問題:目前的產品有哪些？
根據提供的資料，台積公司目前的產品涵蓋智慧型手機、物聯網、車用電子、消費性電子等領域。具體產品包括智慧型手機、物聯網裝置、車用製程技術、消費性電子產品以及其他應用所需的功能。台積公司也在特殊應用積體電路領域取得了重大進展，支援高效能運算及系統單晶片應用，並推出應用於消費性電子產品的人工智慧智能元件。
_________
輸入問題:目前正在開發的項目是?
目前正在開發的項目包括N6 RF+技術、4奈米FinFET強效版（4nm FinFET Plus, N4P）技術、以及在大於90毫米乘90毫米的基板上整合多個系統的技術。此外，還有成功完成全球首個三片晶圓堆疊全局曝光影像感測器技術的開發。
_________
輸入問題:未來一年的計畫？
未來一年的計畫包括持續推動低碳製造，邁向淨零排放，並強化氣候韌性，力成為全球綠色製造的領導者。此外，台積公司也將持續致力於優化製造營運，提高效率和生產力，並增加台灣以外的產能以擴大未來成長潛力。另外，台積公司也將持續進行大學合作計劃，推動半導體技術的研發與創新。
_________
輸入問題: 


## 年報總結與分析

### 1️⃣1️⃣ 匯入 LangChain 總結模組

In [ ]:
from langchain.chains.summarize import load_summarize_chain  # 總結模組

### 方法一：建立問題關鍵字

### 1️⃣2️⃣ 總結原始資料

In [ ]:
# 建立關鍵字串列
key_word = ['有關市場策略的調整或變化有何提及？',
          '公司對未來一年的展望是什麼？',
          '公司的總收入是否增長，淨利潤的正負情況是否有變化？',
          '國際競爭及海外市場情況',
          '目前的研發狀況?']

data_list = []
for word in key_word:
    data = db.max_marginal_relevance_search(word)
    # 整合 Document 串列
    data_list += data

# 建立提示訊息串列
prompt_template = [("system",
                    "以下資料為公司的年報資訊, 你的任務是生成年報摘要及分析報告。\n\
                     年報資訊：{text},\
                     請保留重點如營收漲跌、開發項目等,\
                     最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

### 1️⃣3️⃣  呼叫函式

In [ ]:
chain_refine_16k = load_summarize_chain(llm=llm_16k,
                                        chain_type='stuff',
                                        prompt=prompt)
print(chain_refine_16k.run(data_list))

根據年報資訊，本公司在環境、社會、公司治理方面有以下重大性議題：

1. 公司在環境管理方面有相應的制度，並依據法規進行管理。公司持續致力於排碳減量，推動低碳製造，並強化氣候韌性，以符合氣候變遷因應法的相關要求。
2. 公司積極參與氣候風險辨識與評估，並將其納入企業風險管理流程，以評估重大氣候風險對公司帶來的潛在營運與財務衝擊。
3. 公司在社會方面推動多元文化融合，支持女性在科技領域的發展，並且透過各項計劃和活動，提升年輕世代的孝道意識。

此外，公司也在財務狀況與經營結果方面取得了一些成就，包括營收增長、研發投入、市場規模擴大等。

總的來說，公司在氣候變遷因應、風險管理、社會責任和財務狀況方面都有相應的政策和措施，並且取得了一些成就。未來，公司將持續關注氣候變遷相關法規的更新，並與利害關係人共同努力，以維持長期的永續發展。


### 方法二：關鍵字聯想

### 1️⃣4️⃣  提取關鍵字

In [ ]:
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

word_prompt=ChatPromptTemplate.from_messages(messages=[
    ("human","從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容"
    "")]
)
word_chain = LLMChain(llm=llm_16k, prompt=word_prompt)
output_parser.parse(word_chain('公司的營收狀況如何？')['text'])

['成長', '利潤率', '市場份額', '現金流量']

### 1️⃣5️⃣ 設定 AI 角色讓其依據關鍵字來進行分析報告

In [ ]:
data_prompt=ChatPromptTemplate.from_messages(messages=[
    ("system","你現在是一位專業的股票分析師,"
    "你會以詳細、嚴謹的角度進行年報分析, 針對{output}進行分析並提及重要數字\
    ,然後生成一份專業的趨勢分析報告。"),
    ("human","{text}")])

data_chain = LLMChain(llm=llm_16k, prompt=data_prompt)

### 1️⃣6️⃣ 整合函式

In [ ]:
def analyze_chain(input):
    # 搜尋「問題」的相關資料
    data = db.max_marginal_relevance_search(input)

    # 第一個 Chain 元件, 建立「關鍵字」串列
    word = word_chain(input)
    word_list = output_parser.parse(word['text'])

    # 搜尋「關鍵字」的相關資料
    for i in word_list:
      data += db.max_marginal_relevance_search(i,k=2)
    word_list.append(input)

    # 第二個 Chain 元件, 生成分析報告
    result = data_chain({'output':word_list,'text':data})

    return result['text']

### 1️⃣7️⃣ 呼叫函式

In [ ]:
input = '公司的營收狀況如何？'
print(analyze_chain(input))